In [1]:
import tensorflow as tf
import os

In [2]:
"""
directory = "data"
filenames = [f for f in os.listdir(directory) if f.endswith(".tfrecord")]
os.chdir(directory)
raw_dataset = tf.data.TFRecordDataset(filenames)
for raw_record in raw_dataset.take(5):
    example = tf.train.SequenceExample.FromString(raw_record.numpy())
    print(example.context.feature['population_size'])
    print(example.context.feature['selection_coefficient'])

    #example = tf.train.SequenceExample()
    #example.ParseFromString(raw_record.numpy())
    #print(example.context.feature['population_size'])
    #print(example.feature_lists.feature_list['raw_trait_frequencies'].feature[0])

os.chdir("/home/joshua/projects/timeseries-inference/")
len(filenames)
""";

In [3]:
filepath_dataset = tf.data.TFRecordDataset.list_files("data/*.tfrecord", shuffle=True)
dataset = filepath_dataset.interleave(
    lambda x: tf.data.TFRecordDataset(x),
    cycle_length=3,
    block_length=2,
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    deterministic=False)

shuffle_size = 100
batch_size=8

dataset = dataset.shuffle(shuffle_size).prefetch(tf.data.experimental.AUTOTUNE)

#dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

for record in dataset.take(1):
    example = tf.train.SequenceExample.FromString(record.numpy())
    for i in range(2):
        #print(repr(example.feature_lists.feature_list['raw_trait_frequencies'].feature[i]))
        print(repr(example.context.feature['population_size']))

int64_list {
  value: 480
}

int64_list {
  value: 480
}



In [6]:
filepath_dataset = tf.data.TFRecordDataset.list_files("data/*.tfrecord", shuffle=True)
dataset = filepath_dataset.interleave(
    lambda x: tf.data.TFRecordDataset(x),
    cycle_length=1,
    block_length=1,
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    deterministic=False)

context_features = {"population_size" : tf.io.FixedLenFeature((), tf.int64),
                   "selection_coefficient" : tf.io.FixedLenFeature((), tf.float32)}
sequence_features = {"raw_trait_frequencies" : tf.io.RaggedFeature(tf.float32)}
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE).repeat(10)
#dataset = dataset.batch(2).prefetch(tf.data.experimental.AUTOTUNE)

sample_size = 1 # need to figure out how to increase this and keep keras happy
#for record in dataset.batch(30):
    #example = tf.io.parse_sequence_example(record, context_features=context_features, sequence_features=sequence_features)
    #print(example[0]['population_size'])

pop_size_min = 20
pop_size_max = 500
sc_min = -0.1
sc_max = 1.

def preprocess(record):
    example = tf.io.parse_sequence_example(record, context_features=context_features, sequence_features=sequence_features)
    features = tf.gather(example[1]["raw_trait_frequencies"], tf.random.uniform([sample_size], 0, 10, dtype=tf.int32))
    # scale population size and selection coefficient between 0 and 1 (both are uniformly distributed)
    pop_size_norm = (tf.cast(example[0]["population_size"], dtype=tf.float32) - pop_size_min) / (pop_size_max - pop_size_min)
    sc_norm = (tf.cast(example[0]["selection_coefficient"], dtype=tf.float32) - sc_min) / (sc_max - sc_min)
    labels = [pop_size_norm, sc_norm]
    #labels = [ tf.cast(example[0]["population_size"], dtype=tf.float32), example[0]["selection_coefficient"] ]
    return features, labels

"""
def test(dataset, sample_size):
    for record in dataset:
        example = tf.io.parse_sequence_example(record, context_features=context_features, sequence_features=sequence_features)
        features = tf.gather(example[1]["raw_trait_frequencies"], tf.random.uniform([sample_size], 0, 10, dtype=tf.int32))
        labels = [ tf.cast(example[0]["population_size"], dtype=tf.float32), example[0]["selection_coefficient"] ]
    return features, labels
"""
"""
        for i in range(1):
        #print(repr(example.feature_lists.feature_list['raw_trait_frequencies'].feature[i]))
        #print(repr(example.context.feature['population_size']))
        print(repr(example[0]))
        rn = tf.random.uniform([3], 0, 10, dtype=tf.int32)
        print(rn)
        print(repr(tf.gather(example[1]["raw_trait_frequencies"], rn)))
        
        #print(example[0]["selection_coefficient"])

        #print(example[1]["raw_trait_frequencies"].values)
        #print(dir(example[1]["raw_trait_frequencies"]))
        #print(example[1]["raw_trait_frequencies"].indices)
        #print(repr(example[1]["raw_trait_frequencies"][0]))
        
        print(repr(example[1]["raw_trait_frequencies"][rn[0]]))
        print(repr(example[1]["raw_trait_frequencies"][rn[1]]))
        print(repr(example[1]["raw_trait_frequencies"][rn[2]]))

        #print(example[1]["raw_trait_frequencies"][0][0])

        #print(example[1]["raw_trait_frequencies"][0][1])
        
        #print(example[1]["raw_trait_frequencies"][1][0])
        #print(example[1]["raw_trait_frequencies"][1][1])

        #for j in example[1]:
       #     print(j)
""";

In [7]:
dataset = dataset.map(preprocess)
dataset = dataset.batch(10)


In [8]:
from tensorflow import keras
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(None, 1), ragged=True))
model.add(keras.layers.LSTM(50, activation='tanh'))
model.add(keras.layers.Dense(2, activation='linear'))

In [9]:
def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

def abs_dist(y_true, y_pred):
    abs_difference = tf.math.abs(y_true[0] - y_pred[0])
    return tf.reduce_mean(abs_difference, axis=-1)

optimizer = tf.keras.optimizers.Adam(0.002)
model.compile(loss="mse", optimizer=optimizer, metrics=[abs_dist])
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [11]:
history = model.fit(dataset, epochs=25, callbacks=[callback])

Epoch 1/25
30/30 [==============================] - 0s 4ms/step - loss: 0.0520 - abs_dist: 0.1944 - lr: 1.2257e-07
Epoch 2/25
30/30 [==============================] - 0s 4ms/step - loss: 0.0520 - abs_dist: 0.1934 - lr: 1.2257e-07
Epoch 3/25
30/30 [==============================] - 0s 4ms/step - loss: 0.0520 - abs_dist: 0.1642 - lr: 1.2257e-07
Epoch 4/25
30/30 [==============================] - 0s 4ms/step - loss: 0.0520 - abs_dist: 0.1901 - lr: 1.1090e-07
Epoch 5/25
30/30 [==============================] - 0s 4ms/step - loss: 0.0520 - abs_dist: 0.1716 - lr: 1.0035e-07
Epoch 6/25
30/30 [==============================] - 0s 4ms/step - loss: 0.0520 - abs_dist: 0.1782 - lr: 9.0799e-08
Epoch 7/25
30/30 [==============================] - 0s 4ms/step - loss: 0.0520 - abs_dist: 0.2036 - lr: 8.2159e-08
Epoch 8/25
30/30 [==============================] - 0s 4ms/step - loss: 0.0519 - abs_dist: 0.1917 - lr: 7.4340e-08
Epoch 9/25
30/30 [==============================] - 0s 4ms/step - loss: 0.0520 -

In [47]:
#dataset1 = test(dataset, 3)
#print(features)
#print(labels)
dataset1 = dataset.map(test)
for record in dataset1.take(1):
    print(tf.size(record[0]))

NameError: name 'test' is not defined

In [ ]:
def test(dataset, sample_size):
    for record in dataset.batch(3):
        example = tf.io.parse_sequence_example(record, context_features=context_features, sequence_features=sequence_features)
        features = tf.gather(example[1]["raw_trait_frequencies"], tf.random.uniform([sample_size], 0, 10, dtype=tf.int32))
        labels = [ tf.cast(example[0]["population_size"], dtype=tf.float32), example[0]["selection_coefficient"] ]
    return features, labels


In [34]:
os.chdir("/home/joshua/projects/timeseries-inference/")
